In [ ]:
# Nikhil Patil
# CSEC 620 Project 4
# NLP and SMS Spam 

# Imports 
import pandas as pd
import numpy as np
import re, warnings, nltk, random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
warnings.filterwarnings("ignore")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
sms = pd.read_csv('SMSSpamCollection', sep="\t", names=['label', 'text'])
print("\n\n",sms.head(), end='\n\n')
print(sms.info(), end='\n\n')
print(sms.describe(), end='\n\n')
print("The number of ham (non-spam) messages present are:",sms['label'].value_counts()[0])
print("The number of spam messages present are:",sms['label'].value_counts()[1])

In [ ]:
def tokenize(text):
    '''
    Simple text tokenizer that is used with the Term Frequency Inverse Document Frequency Vectorizer.\n 
    The way this function works is that it takes stop words from the Natural Language Toolkit NTLK[https://www.nltk.org/] 
    Passes them into a regex and uses WordNetLemmatizer to tokenize the text.
    :param text: to be tokenized
    :return: text tokens
    '''    
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)

    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [ ]:
all_words = TfidfVectorizer(tokenizer=tokenize)

In [ ]:
spam_word_counts = all_words.fit_transform(sms.loc[sms['label']=='spam']['text'].values);

spam_words = all_words.vocabulary_

spam_words_df = pd.DataFrame.from_dict(spam_words, orient='index')
spam_words_df.reset_index(inplace=True)
spam_words_df.columns = ['word', 'counts']

spam_words_df.sort_values(by='counts', ascending=False, inplace=True)
spam_words_df.head(15)

ham_word_counts = all_words.fit_transform(sms.loc[sms['label']=='ham']['text'].values)

ham_words = all_words.vocabulary_

ham_words_df = pd.DataFrame.from_dict(ham_words, orient='index')
ham_words_df.reset_index(inplace=True)
ham_words_df.columns = ['word', 'counts']

ham_words_df.sort_values(by='counts', ascending=False, inplace=True)
ham_words_df.head(15)

In [ ]:
tfidf = TfidfTransformer(smooth_idf=False)

spam_word_weights = tfidf.fit_transform(spam_word_counts)
top_idf_indices = tfidf.idf_.argsort()[:-5:-1]
ind_to_word = all_words.get_feature_names_out()
print('The most significant words for spam messages are:')
for ind in top_idf_indices:
    print(tfidf.idf_[ind], ind_to_word[ind])
    
ham_word_weights = tfidf.fit_transform(ham_word_counts)
top_idf_indices = tfidf.idf_.argsort()[:-5:-1]
ind_to_word = all_words.get_feature_names_out()
print('The most significant words for ham messages are:')
for ind in top_idf_indices:
    print(tfidf.idf_[ind], ind_to_word[ind])

In [ ]:
class KMeans:
    """
    A simple K-means clustering implementation.

    This class implements the K-means clustering algorithm to group data points into `k` clusters
    based on their distance to the cluster centroids. The centroids are initialized randomly, and
    the algorithm iterates to minimize the within-cluster variance.

    Attributes:
        k (int): The number of clusters to form.
        max_iters (int): The maximum number of iterations for the algorithm to converge. Default is 100.
        centroids (np.ndarray): The coordinates of the cluster centroids.
        clusters (np.ndarray): The cluster labels for each data point.
    
    Methods:
        euclidean_distance(point1, point2): Computes the Euclidean distance between two data points.
        fit(X): Fits the K-means model to the data by determining the cluster centroids and assigning data points to clusters.
        predict(X): Predicts the nearest cluster for a new set of data points based on the fitted centroids.
        create_clusters(X): Assigns data points to the nearest cluster based on the current centroids.
        calculate_centroids(X): Calculates the new centroids based on the data points in each cluster.
    """

    def __init__(self, k=2, max_iters=100):
        self.k = k
        self.max_iters = max_iters
        self.centroids = None
        self.clusters = None

    def euclidean_distance(self, point1, point2):
        return np.sqrt(np.sum((point1 - point2) ** 2))

    def fit(self, X):

        np.random.seed(88) 
        random_indices = np.random.permutation(X.shape[0])[:self.k]
        self.centroids = X[random_indices]

        for i in range(self.max_iters):
            self.clusters = self.create_clusters(X)
            old_centroids = self.centroids
            self.centroids = self.calculate_centroids(X)
            if np.all(old_centroids == self.centroids):
                break

    def create_clusters(self, X):
        clusters = [[] for _ in range(self.k)]
        for idx, point in enumerate(X):
            closest_centroid = np.argmin([self.euclidean_distance(point, centroid) for centroid in self.centroids])
            clusters[closest_centroid].append(idx)
        return clusters

    def calculate_centroids(self, X):
        centroids = np.zeros((self.k, X.shape[1]))
        for cluster_idx, cluster in enumerate(self.clusters):
            if cluster:  # Avoid empty clusters
                centroids[cluster_idx] = np.mean(X[cluster], axis=0)
        return centroids

    def predict(self, X):
        return np.array([np.argmin([self.euclidean_distance(x, centroid) for centroid in self.centroids]) for x in X])

In [ ]:
class KNearestNeighbors:
    '''
    K-Nearest Neighbors Implementation

    Parameters: 
    k (int) : Number of components 

    Methods:
    fit(X, y): Fit the model using X as training data and y as target values
    predict(X): Predict the class labels for the provided data
    _predict(x): Predict the class label for a single data point

    '''
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)
    
    def _predict(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [ ]:
# data preprocessing 

word_counts = all_words.fit_transform(sms['text'].values)

sms['label'] = sms['label'].map({'spam': 1, 'ham': 0})
y = sms['label'].values

tfidf_weights = tfidf.fit_transform(word_counts)

X = tfidf_weights.toarray()

In [ ]:
# K Nearest Neighbors 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_test_encoded = label_encoder.fit_transform(y_test)


knn = KNearestNeighbors(k=3)
knn.fit(X_train, y_encoded)

predictions = knn.predict(X_test)

accuracy = np.sum(predictions == y_test) / len(y_test)
print(f"KNN Model Accuracy:{accuracy * 100:.2f}%")

print("Classification Report for K-Nearest Neighbors (KNN):")
print(classification_report(y_test_encoded, predictions, target_names=['ham', 'spam']))

conf_matrix = confusion_matrix(y_test, predictions)
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Actual Ham', 'Actual Spam'], columns=['Predicted Ham', 'Predicted Spam'])
print("Confusion Matrix:")
print(conf_matrix_df)

In [ ]:
# K-means 
k = 4
kmeans = KMeans(k=k)
kmeans.fit(X)

predicted_clusters = kmeans.predict(X)

cluster_to_label = {}

for cluster in range(k):
    cluster_indices = np.where(predicted_clusters == cluster)[0]
    
    if len(cluster_indices) > 0: 
        most_common_label = np.bincount(y[cluster_indices]).argmax()
        cluster_to_label[cluster] = most_common_label

mapped_predictions = np.array([cluster_to_label[cluster] for cluster in predicted_clusters])
accuracy = accuracy_score(y, mapped_predictions)
print(f"K-means Clustering Accuracy: {accuracy * 100:.2f}%")
print("Classification Report for K-Means:")
print(classification_report(y, mapped_predictions, target_names=['ham', 'spam']))
conf_matrix = confusion_matrix(y, predicted_clusters)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Cluster 0', 'Cluster 1','Cluster 2','Cluster 3'], yticklabels=['Actual Ham', 'Actual Spam'])
plt.xlabel("Predicted Clusters")
plt.ylabel("Actual Labels")
plt.title("Confusion Matrix for K-means Clustering")
plt.show()